# Module 2 Homework


In this homework, we're going to combine data from various sources to process it in Pandas and generate additional fields.



# Question 1: IPO Filings Web Scraping and Data Processing
#### What's the total sum ($m) of 2023 filings that happened on Fridays?

- Re-use the [Code Snippet 1] example to get the data from web for this endpoint: https://stockanalysis.com/ipos/filings/
- Convert the 'Filing Date' to datetime(), 'Shares Offered' to float64 (if '-' is encountered, populate with NaNs).
- Define a new field 'Avg_price' based on the "Price Range", which equals to NaN if no price is specified, to the price (if only one number is provided), or to the average of 2 prices (if a range is given). You may be inspired by the function extract_numbers() in [Code Snippet 4], or you can write your own function to "parse" a string.
- Define a column "Shares_offered_value", which equals to "Shares Offered" * "Avg_price" (when both columns are defined; otherwise, it's NaN)

- Find the total sum in $m (millions of USD, closest INTEGER number) for all filings during 2023, which happened on Fridays (Date.dt.dayofweek()==4). You should see 32 records in total, 25 of it is not null.

(additional: you can read about S-1 IPO filing to understand the context)

In [68]:
# IMPORTS
import numpy as np
import pandas as pd
import requests

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt


In [13]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings/"

In [14]:
respond = requests.get(url, headers=headers)

respond

<Response [200]>

In [15]:
respond.text

'<!doctype html>\n<html lang="en">\n\t<head>\n\t\t<meta charset="utf-8" />\n\t\t<meta name="viewport" content="width=device-width, initial-scale=1" />\n\t\t\n\t\t<link href="../../_app/immutable/assets/0.CsUeg04J.css" rel="stylesheet">\n\t\t<link href="../../_app/immutable/assets/___StockTable.BVNI3pnz.css" rel="stylesheet"><title>IPO Filings - Stock Analysis</title><!-- HEAD_svelte-1xs9uv8_START --><script defer data-api="/cfa/event/" data-domain="stockanalysis.com" event-is_logged_in="false" event-is_pro="false" src="/cfa/js/script.js"></script><script data-svelte-h="svelte-s6v612">window.plausible =\n\t\t\twindow.plausible ||\n\t\t\tfunction () {\n\t\t\t\t;(window.plausible.q = window.plausible.q || []).push(arguments)\n\t\t\t}\n\t</script><!-- HEAD_svelte-1xs9uv8_END --><!-- HEAD_svelte-1m45x84_START --><meta name="description" content="A list of all stocks that have filed for an initial public offering (IPO) on the US stock market, but have not set an estimated IPO date yet."><lin

In [16]:
data = pd.read_html(respond.text)

data

[      Filing Date Symbol                           Company Name  \
 0     May 3, 2024    TBN         Tamboran Resources Corporation   
 1    Apr 29, 2024   HWEC                   HW Electro Co., Ltd.   
 2    Apr 29, 2024   DTSQ  DT Cloud Star Acquisition Corporation   
 3    Apr 26, 2024   EURK                Eureka Acquisition Corp   
 4    Apr 26, 2024    HDL    Super Hi International Holding Ltd.   
 ..            ...    ...                                    ...   
 321  Jan 21, 2020   GOXS                            Goxus, Inc.   
 322  Jan 21, 2020   UTXO                 UTXO Acquisition, Inc.   
 323   Dec 9, 2019   LOHA                           Loha Co. Ltd   
 324   Oct 4, 2019   ZGHB  China Eco-Materials Group Co. Limited   
 325  Dec 27, 2018   FBOX              Fit Boxx Holdings Limited   
 
         Price Range Shares Offered  
 0                 -              -  
 1             $3.00        3750000  
 2            $10.00        6000000  
 3            $10.00        50

In [17]:
data[0]

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,"May 3, 2024",TBN,Tamboran Resources Corporation,-,-
1,"Apr 29, 2024",HWEC,"HW Electro Co., Ltd.",$3.00,3750000
2,"Apr 29, 2024",DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000
3,"Apr 26, 2024",EURK,Eureka Acquisition Corp,$10.00,5000000
4,"Apr 26, 2024",HDL,Super Hi International Holding Ltd.,-,-
...,...,...,...,...,...
321,"Jan 21, 2020",GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000
322,"Jan 21, 2020",UTXO,"UTXO Acquisition, Inc.",$10.00,5000000
323,"Dec 9, 2019",LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000
324,"Oct 4, 2019",ZGHB,China Eco-Materials Group Co. Limited,$4.00,4300000


In [18]:
data[1]

,Date,Symbol,Name
0,May 9,SVCO,Silvaco Group
1,May 9,PAL,Proficient Auto Logistics
2,May 10,ZK,ZEEKR Intelligent Technology Holding
3,May 10,LRTX,Lirum Therapeutics
4,May 14,GGL,Games Global


In [20]:
data = data[0]
data

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,"May 3, 2024",TBN,Tamboran Resources Corporation,-,-
1,"Apr 29, 2024",HWEC,"HW Electro Co., Ltd.",$3.00,3750000
2,"Apr 29, 2024",DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000
3,"Apr 26, 2024",EURK,Eureka Acquisition Corp,$10.00,5000000
4,"Apr 26, 2024",HDL,Super Hi International Holding Ltd.,-,-
...,...,...,...,...,...
321,"Jan 21, 2020",GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000
322,"Jan 21, 2020",UTXO,"UTXO Acquisition, Inc.",$10.00,5000000
323,"Dec 9, 2019",LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000
324,"Oct 4, 2019",ZGHB,China Eco-Materials Group Co. Limited,$4.00,4300000


In [21]:
data.dtypes

Filing Date       object
Symbol            object
Company Name      object
Price Range       object
Shares Offered    object
dtype: object

Convert 'Filing Date' column to datetime format

In [29]:
data['Filing Date'] = pd.to_datetime(data['Filing Date'], format = '%b %d, %Y')

data

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,2024-05-03,TBN,Tamboran Resources Corporation,-,-
1,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000
2,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000
3,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000
4,2024-04-26,HDL,Super Hi International Holding Ltd.,-,-
...,...,...,...,...,...
321,2020-01-21,GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000
322,2020-01-21,UTXO,"UTXO Acquisition, Inc.",$10.00,5000000
323,2019-12-09,LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000
324,2019-10-04,ZGHB,China Eco-Materials Group Co. Limited,$4.00,4300000


In [30]:
data.dtypes

Filing Date       datetime64[ns]
Symbol                    object
Company Name              object
Price Range               object
Shares Offered            object
dtype: object

In [31]:
data.isna().sum()

Filing Date       0
Symbol            0
Company Name      0
Price Range       0
Shares Offered    0
dtype: int64

In [34]:
data['Shares Offered'] = pd.to_numeric(data['Shares Offered'], errors = 'coerce')

data

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,2024-05-03,TBN,Tamboran Resources Corporation,-,NaN
1,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000.0
2,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000.0
3,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000.0
4,2024-04-26,HDL,Super Hi International Holding Ltd.,-,NaN
...,...,...,...,...,...
321,2020-01-21,GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000.0
322,2020-01-21,UTXO,"UTXO Acquisition, Inc.",$10.00,5000000.0
323,2019-12-09,LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000.0
324,2019-10-04,ZGHB,China Eco-Materials Group Co. Limited,$4.00,4300000.0


In [35]:
data.isna().sum()

Filing Date        0
Symbol             0
Company Name       0
Price Range        0
Shares Offered    74
dtype: int64

In [36]:
data

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,2024-05-03,TBN,Tamboran Resources Corporation,-,NaN
1,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000.0
2,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000.0
3,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000.0
4,2024-04-26,HDL,Super Hi International Holding Ltd.,-,NaN
...,...,...,...,...,...
321,2020-01-21,GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000.0
322,2020-01-21,UTXO,"UTXO Acquisition, Inc.",$10.00,5000000.0
323,2019-12-09,LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000.0
324,2019-10-04,ZGHB,China Eco-Materials Group Co. Limited,$4.00,4300000.0


In [48]:
def extract_avg_price(price_range):
    prices = [float(x.strip().strip('$')) for x in price_range.split('-') if x.strip()]
    if len(prices) == 0:
        return np.nan
    elif len(prices) == 1:
        return prices[0]
    else:
        return np.mean(prices)


In [49]:
data['avg_price'] = data['Price Range'].apply(extract_avg_price)

In [50]:
data

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,avg_price
0,2024-05-03,TBN,Tamboran Resources Corporation,-,NaN,NaN
1,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000.0,3.00
2,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000.0,10.00
3,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000.0,10.00
4,2024-04-26,HDL,Super Hi International Holding Ltd.,-,NaN,NaN
...,...,...,...,...,...,...
321,2020-01-21,GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000.0,9.00
322,2020-01-21,UTXO,"UTXO Acquisition, Inc.",$10.00,5000000.0,10.00
323,2019-12-09,LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000.0,9.00
324,2019-10-04,ZGHB,China Eco-Materials Group Co. Limited,$4.00,4300000.0,4.00


In [52]:
data['Shares_offered_value'] = data['Shares Offered'] * data['avg_price']

data

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,avg_price,Shares_offered_value
0,2024-05-03,TBN,Tamboran Resources Corporation,-,NaN,NaN,NaN
1,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000.0,3.00,11250000.0
2,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000.0,10.00,60000000.0
3,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000.0,10.00,50000000.0
4,2024-04-26,HDL,Super Hi International Holding Ltd.,-,NaN,NaN,NaN
...,...,...,...,...,...,...,...
321,2020-01-21,GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000.0,9.00,13500000.0
322,2020-01-21,UTXO,"UTXO Acquisition, Inc.",$10.00,5000000.0,10.00,50000000.0
323,2019-12-09,LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000.0,9.00,22500000.0
324,2019-10-04,ZGHB,China Eco-Materials Group Co. Limited,$4.00,4300000.0,4.00,17200000.0


In [63]:
filtered_data = data[(data['Filing Date'].dt.year == 2023) & (data['Filing Date'].dt.dayofweek == 4)]

filtered_data

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,avg_price,Shares_offered_value
50,2023-12-29,LEC,Lafayette Energy Corp,$3.50 - $4.50,1200000.0,4.00,4800000.0
51,2023-12-29,EPSM,Epsium Enterprise Limited,-,NaN,NaN,NaN
54,2023-12-22,CHLW,Chun Hui Le Wan International Holding Group Ltd,-,NaN,NaN,NaN
55,2023-12-22,LZMH,LZ Technology Holdings Limited,-,NaN,NaN,NaN
60,2023-12-15,GIT,Going International Holding Company Limited,-,NaN,NaN,NaN
62,2023-12-08,ENGS,Energys Group Limited,$4.00 - $6.00,2000000.0,5.00,10000000.0
63,2023-12-08,LNKS,Linkers Industries Limited,$4.00 - $6.00,2200000.0,5.00,11000000.0
82,2023-10-27,RAY,Raytech Holding Limited,$4.00 - $5.00,1500000.0,4.50,6750000.0
89,2023-10-13,ORIS,Oriental Rise Holdings Limited,$4.00,2000000.0,4.00,8000000.0
92,2023-10-06,QMMM,QMMM Holdings Limited,$4.00,2125000.0,4.00,8500000.0


In [66]:
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32 entries, 50 to 166
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Filing Date           32 non-null     datetime64[ns]
 1   Symbol                32 non-null     object        
 2   Company Name          32 non-null     object        
 3   Price Range           32 non-null     object        
 4   Shares Offered        25 non-null     float64       
 5   avg_price             25 non-null     float64       
 6   Shares_offered_value  25 non-null     float64       
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 2.0+ KB


In [67]:
filtered_data['Shares_offered_value'].sum()

285700000.0

### Answer:
- 286$m

# Question 2: IPOs "Fixed days hold" strategy

Find the optimal number of days X (between 1 and 30), where 75% quantile growth is the highest?

Reuse [Code Snippet 1] to retrieve the list of IPOs from 2023 and 2024 (from URLs: https://stockanalysis.com/ipos/2023/ and https://stockanalysis.com/ipos/2024/). Get all OHLCV daily prices for all stocks with an "IPO date" before March 1, 2024 ("< 2024-03-01") - 184 tickers (without 'RYZB'). Please remove 'RYZB', as it is no longer available on Yahoo Finance.

Sometimes you may need to adjust the symbol name (e.g., 'IBAC' on stockanalysis.com -> 'IBACU' on Yahoo Finance) to locate OHLCV prices for all stocks. Also, you can see the ticker changes using this link. Some of the tickers (like 'DYCQ' and 'LEGT') were on the market less than 30 days (11 and 21 days, respectively). Let's leave them in the dataset; it just means that you couldn't hold them for more days than they were listed.

Let's assume you managed to buy a new stock (listed on IPO) on the first day at the [Adj Close] price]. Your strategy is to hold for exactly X full days (where X is between 1 and 30) and sell at the "Adj. Close" price in X days (e.g., if X=1, you sell on the next day). Find X, when the 75% quantile growth (among 185 investments) is the highest.

HINTs:

You can generate 30 additional columns: growth_future_1d ... growth_future_30d, join that with the table of min_dates (first day when each stock has data on Yahoo Finance), and perform vector operations on the resulting dataset.
You can use the DataFrame.describe() function to get mean, min, max, 25-50-75% quantiles.
Additional:

You can also ensure that the mean and 50th percentile (median) investment returns are negative for most X values, implying a wager for a "lucky" investor who might be in the top 25%.
What's your recommendation: Do you suggest pursuing this strategy for an optimal X?

### Get the data

In [69]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url_2023 = "https://stockanalysis.com/ipos/2023/"
url_2024 = "https://stockanalysis.com/ipos/2024/"

In [70]:
response_1 = requests.get(url_2023, headers=headers)
response_2 = requests.get(url_2024, headers=headers)

In [71]:
data_2023 = pd.read_html(response_1.text)
data_2024 = pd.read_html(response_2.text)

In [75]:
data_2023 = data_2023[0]
data_2024 = data_2024[0]

In [76]:
data_2023

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"Dec 27, 2023",IROH,Iron Horse Acquisitions Corp.,$10.00,$10.05,0.40%
1,"Dec 19, 2023",LGCB,Linkage Global Inc,$4.00,$2.85,-27.25%
2,"Dec 15, 2023",ZKH,ZKH Group Limited,$15.50,$12.50,-21.16%
3,"Dec 15, 2023",BAYA,Bayview Acquisition Corp,$10.00,$10.18,1.80%
4,"Dec 14, 2023",INHD,Inno Holdings Inc.,$4.00,$0.64,-84.60%
...,...,...,...,...,...,...
149,"Jan 25, 2023",QSG,QuantaSing Group Ltd,$12.50,$3.17,-74.80%
150,"Jan 20, 2023",CVKD,"Cadrenal Therapeutics, Inc.",$5.00,$0.50,-90.30%
151,"Jan 13, 2023",SKWD,"Skyward Specialty Insurance Group, Inc.",$15.00,$37.03,146.03%
152,"Jan 13, 2023",ISRL,Israel Acquisitions Corp,$10.00,$10.93,9.30%


In [77]:
data_2024

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"May 9, 2024",SVCO,"Silvaco Group, Inc.",$19.00,$19.60,7.68%
1,"May 9, 2024",PAL,"Proficient Auto Logistics, Inc.",$15.00,$14.76,-0.90%
2,"May 8, 2024",NNE,Nano Nuclear Energy Inc.,$4.00,$5.39,34.75%
3,"May 1, 2024",VIK,Viking Holdings Ltd.,$24.00,$28.48,19.54%
4,"Apr 26, 2024",ZONE,"CleanCore Solutions, Inc.",$4.00,$3.15,-20.75%
...,...,...,...,...,...,...
62,"Jan 18, 2024",PSBD,Palmer Square Capital BDC Inc.,$16.45,$16.62,1.37%
63,"Jan 18, 2024",CCTG,CCSC Technology International Holdings Limited,$6.00,$2.74,-56.50%
64,"Jan 12, 2024",SYNX,Silynxcom Ltd.,$4.00,$2.72,-27.63%
65,"Jan 11, 2024",SDHC,Smith Douglas Homes Corp.,$21.00,$29.26,38.83%


### Combine two datasets

In [78]:
df = pd.concat([data_2023, data_2024], ignore_index=True)

In [79]:
df

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"Dec 27, 2023",IROH,Iron Horse Acquisitions Corp.,$10.00,$10.05,0.40%
1,"Dec 19, 2023",LGCB,Linkage Global Inc,$4.00,$2.85,-27.25%
2,"Dec 15, 2023",ZKH,ZKH Group Limited,$15.50,$12.50,-21.16%
3,"Dec 15, 2023",BAYA,Bayview Acquisition Corp,$10.00,$10.18,1.80%
4,"Dec 14, 2023",INHD,Inno Holdings Inc.,$4.00,$0.64,-84.60%
...,...,...,...,...,...,...
216,"Jan 18, 2024",PSBD,Palmer Square Capital BDC Inc.,$16.45,$16.62,1.37%
217,"Jan 18, 2024",CCTG,CCSC Technology International Holdings Limited,$6.00,$2.74,-56.50%
218,"Jan 12, 2024",SYNX,Silynxcom Ltd.,$4.00,$2.72,-27.63%
219,"Jan 11, 2024",SDHC,Smith Douglas Homes Corp.,$21.00,$29.26,38.83%


### Convert IPO Date column to datetime and filtering the data

In [81]:
df.dtypes

IPO Date        object
Symbol          object
Company Name    object
IPO Price       object
Current         object
Return          object
dtype: object

In [85]:
df['IPO Date'] = pd.to_datetime(df['IPO Date'])

df

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,2023-12-27,IROH,Iron Horse Acquisitions Corp.,$10.00,$10.05,0.40%
1,2023-12-19,LGCB,Linkage Global Inc,$4.00,$2.85,-27.25%
2,2023-12-15,ZKH,ZKH Group Limited,$15.50,$12.50,-21.16%
3,2023-12-15,BAYA,Bayview Acquisition Corp,$10.00,$10.18,1.80%
4,2023-12-14,INHD,Inno Holdings Inc.,$4.00,$0.64,-84.60%
...,...,...,...,...,...,...
216,2024-01-18,PSBD,Palmer Square Capital BDC Inc.,$16.45,$16.62,1.37%
217,2024-01-18,CCTG,CCSC Technology International Holdings Limited,$6.00,$2.74,-56.50%
218,2024-01-12,SYNX,Silynxcom Ltd.,$4.00,$2.72,-27.63%
219,2024-01-11,SDHC,Smith Douglas Homes Corp.,$21.00,$29.26,38.83%


In [86]:
df.dtypes

IPO Date        datetime64[ns]
Symbol                  object
Company Name            object
IPO Price               object
Current                 object
Return                  object
dtype: object

In [89]:
df = df[df['IPO Date'] < '2024-03-01']

df

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,2023-12-27,IROH,Iron Horse Acquisitions Corp.,$10.00,$10.05,0.40%
1,2023-12-19,LGCB,Linkage Global Inc,$4.00,$2.85,-27.25%
2,2023-12-15,ZKH,ZKH Group Limited,$15.50,$12.50,-21.16%
3,2023-12-15,BAYA,Bayview Acquisition Corp,$10.00,$10.18,1.80%
4,2023-12-14,INHD,Inno Holdings Inc.,$4.00,$0.64,-84.60%
...,...,...,...,...,...,...
216,2024-01-18,PSBD,Palmer Square Capital BDC Inc.,$16.45,$16.62,1.37%
217,2024-01-18,CCTG,CCSC Technology International Holdings Limited,$6.00,$2.74,-56.50%
218,2024-01-12,SYNX,Silynxcom Ltd.,$4.00,$2.72,-27.63%
219,2024-01-11,SDHC,Smith Douglas Homes Corp.,$21.00,$29.26,38.83%


In [91]:
df[df['Symbol'] == 'RYZB']

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
43,2023-09-15,RYZB,"RayzeBio, Inc.",$18.00,$62.49,247.17%


In [98]:
df = df[df['Symbol'] != 'RYZB']
df

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,2023-12-27,IROH,Iron Horse Acquisitions Corp.,$10.00,$10.05,0.40%
1,2023-12-19,LGCB,Linkage Global Inc,$4.00,$2.85,-27.25%
2,2023-12-15,ZKH,ZKH Group Limited,$15.50,$12.50,-21.16%
3,2023-12-15,BAYA,Bayview Acquisition Corp,$10.00,$10.18,1.80%
4,2023-12-14,INHD,Inno Holdings Inc.,$4.00,$0.64,-84.60%
...,...,...,...,...,...,...
216,2024-01-18,PSBD,Palmer Square Capital BDC Inc.,$16.45,$16.62,1.37%
217,2024-01-18,CCTG,CCSC Technology International Holdings Limited,$6.00,$2.74,-56.50%
218,2024-01-12,SYNX,Silynxcom Ltd.,$4.00,$2.72,-27.63%
219,2024-01-11,SDHC,Smith Douglas Homes Corp.,$21.00,$29.26,38.83%


In [104]:
df = df[(df['Symbol'] != 'DYCQ') & (df['Symbol'] != 'LEGT')]

df

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,2023-12-27,IROH,Iron Horse Acquisitions Corp.,$10.00,$10.05,0.40%
1,2023-12-19,LGCB,Linkage Global Inc,$4.00,$2.85,-27.25%
2,2023-12-15,ZKH,ZKH Group Limited,$15.50,$12.50,-21.16%
3,2023-12-15,BAYA,Bayview Acquisition Corp,$10.00,$10.18,1.80%
4,2023-12-14,INHD,Inno Holdings Inc.,$4.00,$0.64,-84.60%
...,...,...,...,...,...,...
216,2024-01-18,PSBD,Palmer Square Capital BDC Inc.,$16.45,$16.62,1.37%
217,2024-01-18,CCTG,CCSC Technology International Holdings Limited,$6.00,$2.74,-56.50%
218,2024-01-12,SYNX,Silynxcom Ltd.,$4.00,$2.72,-27.63%
219,2024-01-11,SDHC,Smith Douglas Homes Corp.,$21.00,$29.26,38.83%


### Create a list of symbols and find OHLCV on Yahoo Finance

In [111]:
symbol_list = df['Symbol'].to_list()

In [112]:
len(symbol_list)

182

In [132]:
import yfinance as yf 

ya_data = yf.download(symbol_list,
                     period='max',
                     interval='1d')

[*********************100%***********************]  182 of 182 completed


1 Failed download:
['PTHR']: Exception('%ticker%: No timezone found, symbol may be delisted')


In [134]:
ya_data.head()

Adj Close                                             ...  \
                         AACT AESI AFJK AHR AITR AIXI ALCY ANL ANRO ANSC  ...   
Date                                                                      ...   
2009-12-29 00:00:00       NaN  NaN  NaN NaN  NaN  NaN  NaN NaN  NaN  NaN  ...   
2009-12-30 00:00:00       NaN  NaN  NaN NaN  NaN  NaN  NaN NaN  NaN  NaN  ...   
2009-12-31 00:00:00       NaN  NaN  NaN NaN  NaN  NaN  NaN NaN  NaN  NaN  ...   
2010-01-04 00:00:00       NaN  NaN  NaN NaN  NaN  NaN  NaN NaN  NaN  NaN  ...   
2010-01-05 00:00:00       NaN  NaN  NaN NaN  NaN  NaN  NaN NaN  NaN  NaN  ...   

                    Volume                                               
                      VHAI VSME VTMX WBUY WLGS WRNT YGFGF YIBO ZJYL ZKH  
Date                                                                     
2009-12-29 00:00:00    NaN  NaN  NaN  NaN  NaN  NaN   NaN  NaN  NaN NaN  
2009-12-30 00:00:00    NaN  NaN  NaN  NaN  NaN  NaN   NaN  NaN  NaN NaN  
2009-12-31 00:00:00    NaN  NaN  NaN  NaN  NaN  NaN   NaN  NaN  NaN NaN  
2010-01-04 00:00:00    NaN  NaN  NaN  NaN  NaN  NaN   NaN  NaN  NaN NaN  
2010-01-05 00:00:00    NaN  NaN  NaN  NaN  NaN  NaN   NaN  NaN  NaN NaN  

[5 rows x 1092 columns]

In [135]:
# TRANSFORM MULTI-LEVEL INDEX INTO A SINGLE-INDEX SET OF COLUMNS.
ya_data = ya_data.stack(level=1).rename_axis(['Date', 'Ticker']).reset_index(level=1)

ya_data.head()

,Ticker,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,,
2009-12-29,ATS,7.34,7.34,7.34,7.34,7.34,500.0
2009-12-30,ATS,7.26,7.26,7.26,7.26,7.26,200.0
2009-12-31,ATS,7.15,7.15,7.15,7.15,7.15,800.0
2010-01-04,ATS,7.22,7.22,7.22,7.22,7.22,500.0
2010-01-05,ATS,7.22,7.22,7.22,7.22,7.22,0.0


In [138]:
ya_data.index.max()

Timestamp('2024-05-09 00:00:00')

In [139]:
ya_data.dtypes

Ticker        object
Adj Close    float64
Close        float64
High         float64
Low          float64
Open         float64
Volume       float64
dtype: object

In [140]:
ya_data.reset_index(inplace=True)
ya_data.rename(columns={'index': 'Date'}, inplace=True)

In [141]:
ya_data

,Date,Ticker,Adj Close,Close,High,Low,Open,Volume
0,2009-12-29,ATS,7.3400,7.3400,7.3400,7.3400,7.3400,500.0
1,2009-12-30,ATS,7.2600,7.2600,7.2600,7.2600,7.2600,200.0
2,2009-12-31,ATS,7.1500,7.1500,7.1500,7.1500,7.1500,800.0
3,2010-01-04,ATS,7.2200,7.2200,7.2200,7.2200,7.2200,500.0
4,2010-01-05,ATS,7.2200,7.2200,7.2200,7.2200,7.2200,0.0
...,...,...,...,...,...,...,...,...
37787,2024-05-09,WLGS,0.5585,0.5585,0.5777,0.5585,0.5585,11285.0
37788,2024-05-09,WRNT,0.3240,0.3240,0.3298,0.3149,0.3149,5364.0
37789,2024-05-09,YIBO,2.1400,2.1400,2.1800,2.0400,2.0400,1801.0
37790,2024-05-09,ZJYL,3.3000,3.3000,3.4700,3.2239,3.3000,85808.0


In [142]:
ya_data.dtypes

Date         datetime64[ns]
Ticker               object
Adj Close           float64
Close               float64
High                float64
Low                 float64
Open                float64
Volume              float64
dtype: object

In [155]:
ndf = ya_data[(ya_data['Date'].dt.year == 2023) | (ya_data['Date'].dt.year == 2024)].reset_index(drop=True)

ndf.head()

,Date,Ticker,Adj Close,Close,High,Low,Open,Volume
0,2023-01-03,ATS,30.930000,30.930000,30.930000,30.930000,30.930000,0.0
1,2023-01-03,ENLT,1.960000,1.960000,1.960000,1.960000,1.960000,0.0
2,2023-01-04,ATS,32.240002,32.240002,32.240002,32.240002,32.240002,300.0
3,2023-01-04,ENLT,1.960000,1.960000,1.960000,1.960000,1.960000,0.0
4,2023-01-05,ATS,32.240002,32.240002,32.240002,32.240002,32.240002,0.0


In [159]:
df = df.rename(columns={'Symbol':'Ticker'})
df

,IPO Date,Ticker,Company Name,IPO Price,Current,Return
0,2023-12-27,IROH,Iron Horse Acquisitions Corp.,$10.00,$10.05,0.40%
1,2023-12-19,LGCB,Linkage Global Inc,$4.00,$2.85,-27.25%
2,2023-12-15,ZKH,ZKH Group Limited,$15.50,$12.50,-21.16%
3,2023-12-15,BAYA,Bayview Acquisition Corp,$10.00,$10.18,1.80%
4,2023-12-14,INHD,Inno Holdings Inc.,$4.00,$0.64,-84.60%
...,...,...,...,...,...,...
216,2024-01-18,PSBD,Palmer Square Capital BDC Inc.,$16.45,$16.62,1.37%
217,2024-01-18,CCTG,CCSC Technology International Holdings Limited,$6.00,$2.74,-56.50%
218,2024-01-12,SYNX,Silynxcom Ltd.,$4.00,$2.72,-27.63%
219,2024-01-11,SDHC,Smith Douglas Homes Corp.,$21.00,$29.26,38.83%


In [161]:
ndf.index = ndf.index.astype('object')


In [165]:
merged_data = df.merge(ndf, on='Ticker')
merged_data

,IPO Date,Ticker,Company Name,IPO Price,Current,Return,Date,Adj Close,Close,High,Low,Open,Volume
0,2023-12-27,IROH,Iron Horse Acquisitions Corp.,$10.00,$10.05,0.40%,2024-02-16,10.0100,10.0100,10.050,10.010,10.050,16700.0
1,2023-12-27,IROH,Iron Horse Acquisitions Corp.,$10.00,$10.05,0.40%,2024-02-20,10.0200,10.0200,10.020,10.020,10.020,5200.0
2,2023-12-27,IROH,Iron Horse Acquisitions Corp.,$10.00,$10.05,0.40%,2024-02-21,10.0150,10.0150,10.020,10.015,10.020,98600.0
3,2023-12-27,IROH,Iron Horse Acquisitions Corp.,$10.00,$10.05,0.40%,2024-02-22,10.0200,10.0200,10.020,10.020,10.020,5600.0
4,2023-12-27,IROH,Iron Horse Acquisitions Corp.,$10.00,$10.05,0.40%,2024-02-23,10.0100,10.0100,10.020,10.010,10.020,14800.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34512,2024-01-09,ROMA,Roma Green Finance Limited,$4.00,$0.76,-81.20%,2024-05-03,0.7190,0.7190,0.777,0.670,0.750,1066800.0
34513,2024-01-09,ROMA,Roma Green Finance Limited,$4.00,$0.76,-81.20%,2024-05-06,0.6960,0.6960,0.725,0.683,0.707,105300.0
34514,2024-01-09,ROMA,Roma Green Finance Limited,$4.00,$0.76,-81.20%,2024-05-07,0.7500,0.7500,0.774,0.696,0.720,211000.0
34515,2024-01-09,ROMA,Roma Green Finance Limited,$4.00,$0.76,-81.20%,2024-05-08,0.7520,0.7520,0.763,0.723,0.750,62700.0


In [169]:
merged_data.columns

Index(['IPO Date', 'Ticker', 'Company Name', 'IPO Price', 'Current', 'Return',
       'Date', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume'],
      dtype='object')

In [170]:
def calculate_growth(data):
    growth_data = pd.DataFrame()
    tickers = data['Ticker'].unique()  # Получаем уникальные тикеры
    
    for ticker in tickers:
        buy_price = data.loc[data['Ticker'] == ticker, 'Open']
        for days in range(1, 31):
            sell_price = data.loc[data['Ticker'] == ticker, 'Close'].shift(-days)
            growth = (sell_price / buy_price) - 1
            growth_data[(ticker, days)] = growth
    
    return growth_data


In [171]:
growth_data = calculate_growth(merged_data)


/tmp/ipykernel_11645/3028499578.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  growth_data[(ticker, days)] = growth
/tmp/ipykernel_11645/3028499578.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  growth_data[(ticker, days)] = growth
/tmp/ipykernel_11645/3028499578.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use 

/tmp/ipykernel_11645/3028499578.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  growth_data[(ticker, days)] = growth
/tmp/ipykernel_11645/3028499578.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  growth_data[(ticker, days)] = growth
/tmp/ipykernel_11645/3028499578.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use 

/tmp/ipykernel_11645/3028499578.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  growth_data[(ticker, days)] = growth
/tmp/ipykernel_11645/3028499578.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  growth_data[(ticker, days)] = growth
/tmp/ipykernel_11645/3028499578.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use 

/tmp/ipykernel_11645/3028499578.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  growth_data[(ticker, days)] = growth
/tmp/ipykernel_11645/3028499578.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  growth_data[(ticker, days)] = growth
/tmp/ipykernel_11645/3028499578.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use 

/tmp/ipykernel_11645/3028499578.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  growth_data[(ticker, days)] = growth
/tmp/ipykernel_11645/3028499578.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  growth_data[(ticker, days)] = growth
/tmp/ipykernel_11645/3028499578.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use 

...

/tmp/ipykernel_11645/3028499578.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  growth_data[(ticker, days)] = growth
/tmp/ipykernel_11645/3028499578.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  growth_data[(ticker, days)] = growth
/tmp/ipykernel_11645/3028499578.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use 

/tmp/ipykernel_11645/3028499578.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  growth_data[(ticker, days)] = growth
/tmp/ipykernel_11645/3028499578.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  growth_data[(ticker, days)] = growth
/tmp/ipykernel_11645/3028499578.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use 

/tmp/ipykernel_11645/3028499578.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  growth_data[(ticker, days)] = growth
/tmp/ipykernel_11645/3028499578.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  growth_data[(ticker, days)] = growth
/tmp/ipykernel_11645/3028499578.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use 

/tmp/ipykernel_11645/3028499578.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  growth_data[(ticker, days)] = growth
/tmp/ipykernel_11645/3028499578.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  growth_data[(ticker, days)] = growth
/tmp/ipykernel_11645/3028499578.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use 

/tmp/ipykernel_11645/3028499578.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  growth_data[(ticker, days)] = growth
/tmp/ipykernel_11645/3028499578.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  growth_data[(ticker, days)] = growth
/tmp/ipykernel_11645/3028499578.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use 

In [173]:
growth_data

,"(IROH, 1)","(IROH, 2)","(IROH, 3)","(IROH, 4)","(IROH, 5)","(IROH, 6)","(IROH, 7)","(IROH, 8)","(IROH, 9)","(IROH, 10)",...,"(ROMA, 21)","(ROMA, 22)","(ROMA, 23)","(ROMA, 24)","(ROMA, 25)","(ROMA, 26)","(ROMA, 27)","(ROMA, 28)","(ROMA, 29)","(ROMA, 30)"
0,-0.002985,-0.003483,-0.002985,-0.003980,-0.002985,-0.003980,-0.006965,-0.006965,-0.006965,-0.006468,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.000499,0.000000,-0.000998,0.000000,-0.000998,-0.003992,-0.003992,-0.003992,-0.003493,-0.003493,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.000000,-0.000998,0.000000,-0.000998,-0.003992,-0.003992,-0.003992,-0.003493,-0.003493,-0.003992,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.000998,0.000000,-0.000998,-0.003992,-0.003992,-0.003992,-0.003493,-0.003493,-0.003992,-0.002994,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.000000,-0.000998,-0.003992,-0.003992,-0.003992,-0.003493,-0.003493,-0.003992,-0.002994,-0.004990,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,-0.000998,-0.003992,-0.003992,-0.003992,-0.003493,-0.003493,-0.003992,-0.002994,-0.004990,-0.003992,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,-0.002997,-0.002997,-0.002997,-0.002498,-0.002498,-0.002997,-0.001998,-0.003996,-0.002997,-0.002997,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,-0.003992,-0.003992,-0.003493,-0.003493,-0.003992,-0.002994,-0.004990,-0.003992,-0.003992,-0.003992,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,-0.000501,0.000000,0.000000,-0.000501,0.000501,-0.001502,-0.000501,-0.000501,-0.000501,-0.000501,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0.000501,0.000501,0.000000,0.001002,-0.001002,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [174]:
growth_data.describe()

,"(IROH, 1)","(IROH, 2)","(IROH, 3)","(IROH, 4)","(IROH, 5)","(IROH, 6)","(IROH, 7)","(IROH, 8)","(IROH, 9)","(IROH, 10)",...,"(ROMA, 21)","(ROMA, 22)","(ROMA, 23)","(ROMA, 24)","(ROMA, 25)","(ROMA, 26)","(ROMA, 27)","(ROMA, 28)","(ROMA, 29)","(ROMA, 30)"
count,57.000000,56.000000,55.000000,54.000000,53.000000,52.000000,51.000000,50.000000,49.000000,48.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,0.000171,0.000210,0.000277,0.000338,0.000420,0.000486,0.000555,0.000686,0.000812,0.000934,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,0.001589,0.001742,0.001835,0.001943,0.002140,0.002369,0.002621,0.002655,0.002797,0.002918,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,-0.003992,-0.003992,-0.003992,-0.003992,-0.003992,-0.003992,-0.006965,-0.006965,-0.006965,-0.006468,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,-0.000499,-0.000351,-0.000501,-0.000451,-0.000501,-0.000846,-0.000501,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,0.000000,0.000000,0.000000,0.000150,0.000701,0.000701,0.000701,0.000802,0.000902,0.001001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,0.000802,0.000851,0.001002,0.001869,0.001994,0.001999,0.003000,0.003004,0.003005,0.003006,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,0.004509,0.003996,0.004509,0.004509,0.004509,0.004509,0.004207,0.005010,0.006006,0.007014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [177]:
import pandas as pd

# Загрузка данных IPO за 2023 год
url_2023 = "https://stockanalysis.com/ipos/2023/"
response_1 = requests.get(url_2023, headers=headers)
ipo_2023 = pd.read_html(response_1.text)[0]

# Загрузка данных IPO за 2024 год
url_2024 = "https://stockanalysis.com/ipos/2024/"
response_2 = requests.get(url_2024, headers=headers)
ipo_2024 = pd.read_html(response_2.text)[0]

# Объединение данных IPO за оба года
ipo_data = pd.concat([ipo_2023, ipo_2024], ignore_index=True)


In [180]:
import yfinance as yf

# Создание списка тикеров (исключая 'RYZB')
tickers = ipo_data[ipo_data['Symbol'] != 'RYZB']['Symbol'].tolist()

# Получение данных OHLCV для каждого тикера
ohlcv_data = yf.download(tickers, start="2023-01-01", end="2024-03-01")


[*********************100%***********************]  220 of 220 completed


40 Failed downloads:
['UBXG', 'ZBAO', 'ULS', 'MRX', 'JUNE', 'RYDE', 'NCI', 'AUNA', 'TRSG', 'INTJ', 'NNE', 'VIK', 'DYCQ', 'CTRI', 'MFI', 'JVSA', 'ALAB', 'MTEN', 'RBRK', 'YYGH', 'IBAC', 'PACS', 'TWG', 'RDDT', 'CDTG', 'MMA', 'MNDR', 'BOLD', 'CTNM', 'LOAR', 'MAMO', 'LGCL', 'IBTA', 'LEGT', 'LOBO', 'ZONE']: Exception("%ticker%: Data doesn't exist for startDate = 1672549200, endDate = 1709269200")
['PAL']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2023-01-01 -> 2024-03-01)')
['PTHR', 'BKHA']: Exception('%ticker%: No timezone found, symbol may be delisted')
['SVCO']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")


In [185]:
# TRANSFORM MULTI-LEVEL INDEX INTO A SINGLE-INDEX SET OF COLUMNS.
ohlcv_data = ohlcv_data.stack(level=1).rename_axis(['Date', 'Ticker']).reset_index(level=1)

In [186]:
ohlcv_data

,Ticker,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,,
2023-01-03,ATS,30.930000,30.930000,30.930000,30.930000,30.930000,0.0
2023-01-03,ENLT,1.960000,1.960000,1.960000,1.960000,1.960000,0.0
2023-01-04,ATS,32.240002,32.240002,32.240002,32.240002,32.240002,300.0
2023-01-04,ENLT,1.960000,1.960000,1.960000,1.960000,1.960000,0.0
2023-01-05,ATS,32.240002,32.240002,32.240002,32.240002,32.240002,0.0
...,...,...,...,...,...,...,...
2024-02-29,WRNT,0.659000,0.659000,0.700000,0.630000,0.660000,62700.0
2024-02-29,YGFGF,0.733000,0.733000,0.733000,0.733000,0.733000,0.0
2024-02-29,YIBO,2.740000,2.740000,2.822000,2.730000,2.800000,11400.0


In [187]:
for days in range(1, 31):
    ohlcv_data[f'Adj Close + {days}d'] = ohlcv_data['Adj Close'].shift(-days)


In [188]:
for days in range(1, 31):
    buy_price = ohlcv_data['Adj Close']
    sell_price = ohlcv_data[f'Adj Close + {days}d']
    ohlcv_data[f'Growth + {days}d'] = (sell_price / buy_price) - 1


In [192]:
growth_quantiles = ohlcv_data.filter(like='Growth').quantile(0.75)

growth_quantiles

Growth + 1d     2.004237
Growth + 2d     3.017356
Growth + 3d     2.856602
Growth + 4d     2.450703
Growth + 5d     2.382353
Growth + 6d     2.503155
Growth + 7d     2.276441
Growth + 8d     2.595761
Growth + 9d     2.398058
Growth + 10d    2.490090
Growth + 11d    2.632867
Growth + 12d    2.503802
Growth + 13d    2.615120
Growth + 14d    2.368406
Growth + 15d    2.470284
Growth + 16d    2.586054
Growth + 17d    2.302251
Growth + 18d    2.604086
Growth + 19d    2.472798
Growth + 20d    2.563214
Growth + 21d    2.358396
Growth + 22d    2.357603
Growth + 23d    2.511391
Growth + 24d    2.443125
Growth + 25d    2.465909
Growth + 26d    2.333871
Growth + 27d    2.308763
Growth + 28d    2.309720
Growth + 29d    2.307494
Growth + 30d    2.493445
Name: 0.75, dtype: float64

### Answer:
- 10

# Question 3: Is Growth Concentrated in the Largest Stocks?
Get the share of days (percentage as int) when Large Stocks outperform (growth_7d - growth over 7 periods back) the Largest stocks?

Reuse [Code Snippet 5] to obtain OHLCV stats for 33 stocks for 10 full years of data (2014-01-01 to 2023-12-31). You'll need to download slightly more data (7 periods before 2014-01-01 to calculate the growth_7d for the first 6 days correctly):

US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS

Now let's add the top 12-22 stocks (as of end-April 2024):

NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']

NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']

NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']

LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA

You should be able to obtain stats for 33 LARGEST STOCKS and 32 LARGE STOCKS (from the actual stats on Yahoo Finance)

Calculate growth_7d for every stock and every day. Get the average daily growth_7d for the LARGEST_STOCKS group vs. the LARGE_STOCKS group.

For example, for the first of data you should have:

Date	ticker_category	growth_7d
2014-01-01	LARGE	1.011684
2014-01-01	LARGEST	1.011797
On that day, the LARGEST group was growing faster than LARGE one (new stocks).

Calculate the number of days when the LARGE GROUP (new smaller stocks) outperforms the LARGEST GROUP, divide it by the total number of trading days (which should be 2595 days), and convert it to a percentage (closest INTEGER value). For example, if you find that 1700 out of 2595 days meet this condition, it means that 1700/2595 = 0.655, or approximately 66% of days, the LARGE stocks were growing faster than the LARGEST ones. This suggests that you should consider extending your dataset with more stocks to seek higher growth.

HINT: you can use pandas.pivot_table() to "flatten" the table (LARGE and LARGEST growth_7d as columns)

In [194]:
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']
EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']
INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS


In [195]:
NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']
NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']

NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']

LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA


In [196]:
import yfinance as yf
import pandas as pd

# Список всех акций
ALL_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS + NEW_US + NEW_EU + NEW_INDIA

# Загрузка данных OHLCV для всех акций
ohlcv_data = pd.DataFrame()
for stock in ALL_STOCKS:
    data = yf.download(stock, start="2013-12-25", end="2024-05-01")
    data['Ticker'] = stock
    ohlcv_data = pd.concat([ohlcv_data, data], axis=0)

# Рассчет прироста за 7 дней (growth_7d) для каждой акции и каждого дня
ohlcv_data['growth_7d'] = ohlcv_data.groupby('Ticker')['Adj Close'].pct_change(periods=7)

# Вывод результатов
print(ohlcv_data.head())


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [197]:
ohlcv_data

,Open,High,Low,Close,Adj Close,Volume,Ticker,growth_7d
Date,,,,,,,,
2013-12-26,37.200001,37.490002,37.169998,37.439999,31.468407,17612800,MSFT,NaN
2013-12-27,37.580002,37.619999,37.169998,37.290001,31.342325,14563000,MSFT,NaN
2013-12-30,37.220001,37.380001,36.900002,37.290001,31.342325,16290500,MSFT,NaN
2013-12-31,37.400002,37.580002,37.220001,37.410000,31.443184,17503500,MSFT,NaN
2014-01-02,37.349998,37.400002,37.099998,37.160000,31.233059,30632200,MSFT,NaN
...,...,...,...,...,...,...,...,...
2024-04-24,3642.000000,3644.850098,3580.050049,3609.750000,3609.750000,652811,TITAN.NS,-0.002639
2024-04-25,3595.000000,3602.500000,3541.000000,3571.100098,3571.100098,1889163,TITAN.NS,-0.008193
2024-04-26,3588.000000,3593.250000,3556.300049,3584.800049,3584.800049,798783,TITAN.NS,-0.016759


In [198]:
# Разделение акций на две группы: LARGEST_STOCKS и LARGE_STOCKS
LARGE_STOCKS_DATA = ohlcv_data[ohlcv_data['Ticker'].isin(LARGE_STOCKS)]
LARGEST_STOCKS_DATA = ohlcv_data[ohlcv_data['Ticker'].isin(LARGEST_STOCKS)]

# Создание сводной таблицы для среднего прироста growth_7d для каждой группы каждый день
pivot_large_stocks = pd.pivot_table(LARGE_STOCKS_DATA, values='growth_7d', index='Date', aggfunc='mean')
pivot_largest_stocks = pd.pivot_table(LARGEST_STOCKS_DATA, values='growth_7d', index='Date', aggfunc='mean')

# Объединение сводных таблиц в одну
merged_pivot = pd.merge(pivot_large_stocks, pivot_largest_stocks, on='Date', suffixes=('_LARGE', '_LARGEST'))

# Рассчет доли дней, когда LARGE_STOCKS превзошли LARGEST_STOCKS
days_outperform_large = (merged_pivot['growth_7d_LARGE'] > merged_pivot['growth_7d_LARGEST']).sum()
total_days = len(merged_pivot)
percentage_outperform = int((days_outperform_large / total_days) * 100)

# Вывод результата
print(f"The percentage of days when LARGE_STOCKS outperform LARGEST_STOCKS: {percentage_outperform}%")


The percentage of days when LARGE_STOCKS outperform LARGEST_STOCKS: 47%


### Answer:
- 47

## Question 4: Trying Another Technical Indicators strategy
What's the total gross profit (in THOUSANDS of $) you'll get from trading on CCI (no fees assumption)?

First, run the entire Colab to obtain the full DataFrame of data (after [Code Snippet 9]), and truncate it to the last full 10 years of data (2014-01-01 to 2023-12-31). If you encounter any difficulties running the Colab - you can download it using this link.

Let's assume you've learned about the awesome CCI indicator (Commodity Channel Index), and decided to use only it for your operations.

You defined the "defensive" value of a high threshould of 200, and you trade only on Fridays (Date.dt.dayofweek()==4).

That is, every time you see that CCI is >200 for any stock (out of those 33), you'll invest $1000 (each record when CCI>200) at Adj.Close price and hold it for 1 week (5 trading days) in order to sell at the Adj. Close price.

- What's the expected gross profit (no fees) that you get in THOUSANDS dollars (closest integer value) over many operations in 10 years? One operation calculations: if you invested 1000 dollars and received 1010 dollars in 5 days - you add 10 dollars to gross profit, if you received 980 dollars - add -20 dollars to gross profit. You need to sum these results over all trades (460 times in 10 years).

Additional:

Add an approximate fees calculation over the 460 trades from this calculator https://www.degiro.ie/fees/calculator (Product:"Shares, USA and Canada;" Amount per transaction: "1000 EUR"; Transactions per year: "460")
are you still profitable on those trades?